## UNVEILING THE POWER OF BOOKS: AMAZON BEST BOOK SELLERS 2009-2019

In [1]:
#filter warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#import libraries
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.chrome.service import Service as ChromeService
from bs4 import BeautifulSoup
from urllib.request import urlopen
from time import sleep
import pandas as pd
import numpy as np

In [3]:
#since the url has a consistent pattern over the years, we will be looping through urls to extract our data

urls =[] #initialize an empty list to append the urls
years = [str(i) for i in range(2009,2020)]
for year in years:
    urls.append (f"https://www.amazon.com/gp/bestsellers/{year}/books/ref=zg_bsar_cal_ye")
    urls.append(f"https://www.amazon.com/gp/bestsellers/{year}/books/ref=zg_bsar_pg_2_books/ref=zg_bsar_pg_2_books?ie=UTF8&pg=2")

In [38]:
#define a function to extract each elements.

def get_dir(book,year): 
    '''to get the details of each book for each year''' 
    
    import numpy as np
    '''to get the name of price'''
    
    try:
        Ranks = book.find('span', class_='zg-bdg-text').text[1:]
    except Exception as err:
        Ranks = np.nan
    try:
        Title = book.find('div',class_="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").text
    except Exception as err:
        Title = np.nan
    try:
        Authors = book.find('a',class_="a-size-small a-link-child").text
    except Exception as err:
        Authors = np.nan
    try:
        Cover_type = book.find('span',class_="a-size-small a-color-secondary a-text-normal").text
    except Exception as err:
        Cover_type = np.nan
    
    try:
        Price = book.find('span',class_="_cDEzb_p13n-sc-price_3mJ9Z").text[1:]
    except Exception as err:
        Price = np.nan
    try:
        Reviews  = book.find('span',class_="a-size-small").text
    except Exception as err:
        Reviews = np.nan
    try:
        Ratings= book.find('span',class_="a-icon-alt").text[:3] 
    except Exception as err:
        Ratings = np.nan
    Year = year
    
    return[Ranks, Title, Authors, Cover_type, Price, Reviews, Ratings, Year]    


In [40]:
#initialise a list of years
year = [(str(i),str(i)) for i in range(2009,2020)] 
years = [j for i in year for j in i]
years

['2009',
 '2009',
 '2010',
 '2010',
 '2011',
 '2011',
 '2012',
 '2012',
 '2013',
 '2013',
 '2014',
 '2014',
 '2015',
 '2015',
 '2016',
 '2016',
 '2017',
 '2017',
 '2018',
 '2018',
 '2019',
 '2019']

In [6]:
#Initialize an empty list to extract the two pages using webdriver for all the books 2009-2019

all_books = [] 
from selenium.webdriver.chrome.options import Options
for url in urls: 
    
    webpage = url
    driver = webdriver.Chrome()
    
    driver.get(webpage)        
    
    sleep(60)                 
    
    the_soup = BeautifulSoup(driver.page_source, 'html.parser')           
    books = the_soup.find_all(id = 'gridItemRoot')                
    
    all_books.append(books) 
    
    
    
    driver.quit()

In [41]:
year_index = (list(enumerate(years)))
y = year_index
y

[(0, '2009'),
 (1, '2009'),
 (2, '2010'),
 (3, '2010'),
 (4, '2011'),
 (5, '2011'),
 (6, '2012'),
 (7, '2012'),
 (8, '2013'),
 (9, '2013'),
 (10, '2014'),
 (11, '2014'),
 (12, '2015'),
 (13, '2015'),
 (14, '2016'),
 (15, '2016'),
 (16, '2017'),
 (17, '2017'),
 (18, '2018'),
 (19, '2018'),
 (20, '2019'),
 (21, '2019')]

In [42]:
scraped_data = [] 
for i in y:  
    for books in all_books[i[0]]:             
        for book in books:                   
            scraped_data.append(get_dir(book,i[1])) 

In [45]:
#create a dataframe

df_books = pd.DataFrame(scraped_data, columns = [
                         'Ranks',
                         'Title',
                         'Authors',
                         'Book_cover',
                         'Price',
                         'Reviews',
                         'Ratings',
                       'Year'])
df_books.head()

,Ranks,Title,Authors,Book_cover,Price,Reviews,Ratings,Year
0,1,The Lost Symbol (Robert Langdon),Dan Brown,Hardcover,15.41,"29,023",4.3,2009
1,2,The Shack: Where Tragedy Confronts Eternity,William P. Young,Paperback,10.14,"41,129",4.6,2009
2,3,Liberty and Tyranny: A Conservative Manifesto,Mark R. Levin,Hardcover,14.76,"5,341",4.8,2009
3,4,"Breaking Dawn (The Twilight Saga, Book 4)",Stephenie Meyer,Hardcover,19.55,"25,427",4.7,2009
4,5,Going Rogue: An American Life,Sarah Palin,Hardcover,6.30,"1,598",4.6,2009


In [19]:
df_books.to_csv('Amazon Books 2009-2019.csv')